In [ ]:
import pandas as pd
from google.colab import files
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication
import smtplib
import re

#to upload a file from your local system
uploaded = files.upload()
file_path1 = list(uploaded.keys())[0]
df = pd.read_excel(file_path1)

#sorting from whole data to 'Planned order' data
condition = (df['ORDER_TYPE'] == 'Planned order')
filtered_df = df[condition]

#sorted and saved to file_path (sheet_name= FilteredSheet)
file_path = file_path1
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a') as writer:

    filtered_df.to_excel(writer, sheet_name='FilteredSheet', index=False)

#reads the sorted order data
df = pd.read_excel(file_path,sheet_name='FilteredSheet')

# Convert the PO_RELEASE_DATE column to datetime object
df['PO_RELEASE_DATE'] = pd.to_datetime(df['PO_RELEASE_DATE'])

# Extract year and quarter from the date column
df['Year'] = df['PO_RELEASE_DATE'].dt.year
def get_quarter(month):
    if month in [1, 2, 3]:
        return 'Q1'
    elif month in [4, 5, 6]:
        return 'Q2'
    elif month in [7, 8, 9]:
        return 'Q3'
    else:
        return 'Q4'

df['Quarter'] = df['PO_RELEASE_DATE'].dt.month.apply(get_quarter)

# Create a pivot table with Year, Quarter, and SUPPLIER_ITEM
pivot_table = pd.pivot_table(df, values=['REM_QTY'],
                             index=['Year', 'Quarter'], columns=['SUPPLIER_ITEM'], aggfunc='sum')

pivot_table.reset_index(inplace=True)
pivot_table.columns = [col[1] if isinstance(col, tuple) else col for col in pivot_table.columns.values]

#for total sum of req quantity
specific_columns = [2792881, 2792885, 2792886]
pivot_table['Total'] = pivot_table[specific_columns].sum(axis=1)
pivot_table.loc['Total', specific_columns] = pivot_table[specific_columns].sum()

# Replace NaN values with '-'
pivot_table.fillna('-', inplace=True)

# Save the result to a new Excel file
output_file_path = 'pivot_table_output.xlsx'
pivot_table.to_excel(output_file_path,sheet_name="Total_Overview", index=False)


# Load the Excel file into a pandas dataframe
excel_file_path = output_file_path
sheet_name = 'Total_Overview'


df = pd.read_excel(excel_file_path, sheet_name=sheet_name, engine='openpyxl')



# Save the updated dataframe to a new Excel file
df.to_excel(excel_file_path,sheet_name=sheet_name, index=False)

df.rename(columns={
    'Unnamed: 0': 'Year',
    'Unnamed: 1': 'Quater'
}, inplace=True)

# Save the DataFrame back to the Excel file
df.to_excel(excel_file_path,sheet_name=sheet_name, index=False)



df = pd.read_excel(file_path,sheet_name='FilteredSheet')

# Convert the date column to datetime object
df['PO_RELEASE_DATE'] = pd.to_datetime(df['PO_RELEASE_DATE'])

# Extract year, quarter, and month from the date column
df['Year'] = df['PO_RELEASE_DATE'].dt.year

# Map month numbers to month names
month_names = {
    1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr',
    5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug',
    9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'
}

df['Month'] = df['PO_RELEASE_DATE'].dt.month.map(month_names)

def get_quarter(month):
    if month in ['Jan', 'Feb', 'Mar']:
        return 'Q1'
    elif month in ['Apr', 'May', 'Jun']:
        return 'Q2'
    elif month in ['Jul', 'Aug', 'Sep']:
        return 'Q3'
    else:
        return 'Q4'

df['Quarter'] = df['Month'].map(get_quarter)


# Create a pivot table with Year, Quarter, Month, and SUPPLIER_ITEM
pivot_table = pd.pivot_table(df, values='REM_QTY',
                             index=['Year', 'Quarter', 'Month'], columns=['SUPPLIER_ITEM'], aggfunc='sum')


pivot_table.reset_index(inplace=True)
pivot_table.columns = [f'{col[0]}_{col[1]}' if isinstance(col, tuple) else col for col in pivot_table.columns]

specific_columns = [2792881, 2792885, 2792886]
pivot_table['Total'] = pivot_table[specific_columns].sum(axis=1)
pivot_table.loc['Total', specific_columns] = pivot_table[specific_columns].sum()

# Replace NaN values with '-'
pivot_table.fillna('-', inplace=True)

with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a') as writer:
    # Write the pivot table to a new sheet
    pivot_table.to_excel(writer, sheet_name='Month Wise', index=False)

print(f'Data is summarized and saved to {output_file_path}')


source_sheet_name = 'Sheet 1'
df_source = pd.read_excel(file_path1, sheet_name=source_sheet_name)

with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    # Write the source DataFrame to a new sheet in the output file
    df_source.to_excel(writer, sheet_name='Garmin_Data', index=False)


import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.application import MIMEApplication

# Email configuration for Outlook
outlook_smtp_server = 'smtp-mail.outlook.com'
outlook_smtp_port = 587
outlook_sender_email = 'y.e.nithish@outlook.com'
outlook_sender_password = 'nithish@21'

# Recipient email (Gmail account)
recipient_email = ['y.e.nithish@outlook.com','kirankv@titan.co.in']

# Create a message
message = MIMEMultipart()
message['From'] = outlook_sender_email
message['To'] = ', '.join(recipient_email)
message['Subject'] = 'Garmin PO Visibility'


# Attach the Excel file
file_path = output_file_path
xls = pd.ExcelFile(file_path)
html_body = '<html><head></head><body>'

sheet_name="Total_Overview"
df = pd.read_excel(xls, sheet_name=sheet_name)

    # Convert the table to HTML format
table_html = df.to_html(index=False, justify='left', classes='table table-striped table-bordered')

    # Remove ".0" from the end of numbers using regex
table_html = re.sub(r'\.0(?=\s|</td>)', '', table_html)

    # Add the HTML table to the email body
html_body +=f'<p>Dear Sirs,</p><p>Good Day!</p><br><p>Please find the consolidated data of the Garmin PO Visibility sheet.</p>'
html_body += f'<h2>{sheet_name}</h2>'
html_body += table_html
html_body += f'<p><i>*This is an automated email. Kindly revert for any clarifications*</i></p><br>'
html_body += f'<p>Thanks & Regards,</p>Kiran KV'

# Complete the HTML body
html_body += '</body></html>'

# Attach the HTML body to the email
message.attach(MIMEText(html_body, 'html'))




with open(file_path, 'rb') as file:
    part = MIMEApplication(file.read(), Name=file_path)
    part['Content-Disposition'] = f'attachment; filename={file.name}'
    message.attach(part)




try:
    outlook_server = smtplib.SMTP(outlook_smtp_server, outlook_smtp_port)
    outlook_server.starttls()
    outlook_server.login(outlook_sender_email, outlook_sender_password)

    # Send the email from Outlook to Gmail
    outlook_server.sendmail(outlook_sender_email, recipient_email, message.as_string())
    outlook_server.quit()

    print('Email sent successfully from Outlook to Gmail')
except Exception as e:
    print(f'Error sending email from Outlook to Gmail: {str(e)}')


Saving T1166625.xlsx to T1166625 (27).xlsx


/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/usr/local/lib/python3.10/dist-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
<ipython-input-28-d0649f8a61e1>:55: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivot_table.loc['Total', specific_columns] = pivot_table[specific_columns].sum()
<ipython-input-28-d0649f8a61e1>:127: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  pivot_table.loc['Total', specific_columns] = pivot_table[specific_columns].sum()


Data is summarized and saved to pivot_table_output.xlsx
Email sent successfully from Outlook to Gmail
